In [55]:
import os
import random
import zlib
from collections import defaultdict
from itertools import combinations
from typing import Callable

import numpy as np
import pandas as pd
from scipy.optimize import fsolve
from tqdm.auto import tqdm

In [49]:
def is_prime(n: int):
    if n < 2:
        return False
    for i in range(2, int(np.sqrt(n))+1):
        if (n % i) == 0:
            return False
    return True

class HashGenerator:
    def __init__(
        self, 
        num_rows: int = np.iinfo(np.uint32).max, 
        prime: int = 4294967387
    ) -> None:
        assert is_prime(prime)
        assert prime >= num_rows

        self.prime = prime
        self.num_rows = num_rows
        self.a_set = set()
        self.b_set = set()

    def get_num_rows(self) -> int:
        return self.num_rows

    def next(self) -> Callable[[np.uint32], np.uint32]:
        a = self._generate_coeff(self.a_set, self.num_rows)
        b = self._generate_coeff(self.b_set, self.num_rows)
        return lambda row: np.uint32((a * row + b) % self.prime)

    def reset(self) -> None:
        self.a_set = set()
        self.b_set = set()

    def _generate_coeff(
        self, 
        coeff_set: set[int],
        max_val: int
    ) -> int:
        while True:
            coeff = random.randint(1, max_val)
            if coeff not in coeff_set:
                coeff_set.add(coeff)
                return coeff

In [56]:
def jaccard_similarity(
    x: np.ndarray, 
    y: np.ndarray
) -> float:
    numerator = len(set(x).intersection(set(y)))
    denominator = len(set(x).union(set(y)))
    return numerator / denominator

class LSHModel:
    def __init__(
        self,
        k: int,
        threshold: float,
        num_hashes: int,
        hash_generator: HashGenerator,
        shingle_hash: Callable = zlib.crc32
    ) -> None:
        self.k = k
        self.threshold = threshold
        self.num_hashes = num_hashes
        self.hash_generator = hash_generator
        self.num_rows = self.hash_generator.get_num_rows()
        self.shingle_hash = shingle_hash
        self.num_docs = 0
        self.docs_dict = dict()
        self.signature = None
        self.candidate_pairs = set()
        self.fp_pairs = set()
        self.similar_pairs = set()
        self.b = -1
        self.r = -1

    def add_document(self, doc: str) -> None:
        shingles = self._create_shingles(
            doc,
            self.k, 
            self.shingle_hash
        )
        self.docs_dict[self.num_docs] = shingles
        self.num_docs += 1

    def get_similar_pairs(self) -> set[tuple[int, int]]:
        self.hash_generator.reset()
        hash_functions = [
            self.hash_generator.next()
            for _ in range(self.num_hashes)
        ]
        self.signature = self._build_signature(
            self.docs_dict,
            self.num_rows,
            hash_functions
        )
        self.b, self.r = self._find_lsh_params(
            self.threshold,
            self.num_hashes
        )
        self.candidate_pairs = self._lsh(
            self.signature,
            self.b
        )
        self.similar_pairs, self.fp_pairs = \
            self._check_threshold_on_signature(
                self.candidate_pairs,
                self.signature,
                self.threshold
            )
        return self.similar_pairs
        
    def _create_shingles(
        self, 
        doc: str,
        k: int, 
        hash_f: Callable
    ) -> np.ndarray:
        # mmh3.hash
        return np.unique(
            [
                hash_f(str.encode(doc[i:i+k]))
                for i in range(len(doc[:-k+1]))
            ]
        ).astype(np.uint32)

    def _build_signature(
        self,
        docs_dict: dict[int, np.ndarray],
        num_rows: int, 
        hash_functions: list[Callable[[np.uint32], np.uint32]]
    ) -> np.ndarray:
        signature = np.full(
            (len(hash_functions), len(docs_dict)), 
            fill_value=np.inf
        )
        for r in tqdm(range(0, num_rows)):
            hash_values = [
                f(r)
                for f in hash_functions
            ]
            for c, shingles in enumerate(docs_dict.values()):
                if r in shingles:
                    for i, hash_val in enumerate(hash_values):
                        if hash_val < signature[i,c]:
                            signature[i,c] = hash_val 

        return signature.astype(np.uint32)

    def _find_lsh_params(self, t: int, n: int) -> tuple[int]:
        """A lower b means that two items must match a higher
        number of rows. By taking the floor of b, we favor
        more similar pairs.  
        """
        def equations(vars):
            b, r = vars
            eq1 = t - (1 / b) ** (1 / r)
            eq2 = n - b * r
            return [eq1, eq2]

        b, r =  fsolve(equations, (1, 1))
        b = np.floor(b)
        r = n // b
        return int(b), int(r)

    def _lsh(
        self, 
        signature: np.ndarray, 
        b: int
    ) -> set[tuple[int, int]]:
        candidate_pairs = set()
        
        for band in np.array_split(signature, b):
            
            # column tuple -> list of column indices having that tuple
            same_columns = defaultdict(list) 
            
            for c in range(band.shape[1]):
                column = band[:,c]
                same_columns[tuple(column)].append(c)

            filtered_same_columns = dict()
            for k, values in same_columns.items():
                if len(values) >= 2:
                    filtered_same_columns[k] = values

            for values in filtered_same_columns.values():
                for pair in combinations(values, 2):
                    candidate_pairs.add(pair)

        return candidate_pairs

    def _check_threshold_on_signature(
        self, 
        candidate_pairs: list[tuple[int, int]], 
        signature: np.ndarray, 
        t: float
    ) -> tuple[set[tuple[int, int, float]]]:
        similar_pairs = set()
        false_positive_pairs = set()

        for (x, y) in candidate_pairs:
            x_col = signature[:,x]
            y_col = signature[:,y]
            similarity = sum(x_col == y_col) / signature.shape[0]
            tup = (x, y, similarity)
            if similarity >= t:
                similar_pairs.add(tup)
            else:
                false_positive_pairs.add(tup)

        return similar_pairs, false_positive_pairs

    def check_threshold_on_cm(
        self,
        candidate_pairs: list[tuple[int, int]], 
        docs_dict: dict[int, np.ndarray], 
        t: float
    ) -> tuple[set[tuple[int, int, float]]]:
        similar_pairs = set()
        false_positive_pairs = set()

        for (x, y) in candidate_pairs:
            similarity = jaccard_similarity(docs_dict[x], docs_dict[y])
            tup = (x, y, similarity)
            if similarity >= t:
                similar_pairs.add(tup)
            else:
                false_positive_pairs.add(tup)

        return similar_pairs, false_positive_pairs

# Test

In [4]:
dataset_path = r"E:\datasets\ukraine"
os.walk(dataset_path)

<generator object _walk at 0x000001DB898B0C10>

In [7]:
for name in os.listdir(dataset_path):
    if os.path.isfile(os.path.join(dataset_path, name)):
        print(name)

0401_UkraineCombinedTweetsDeduped.csv.gzip
0402_UkraineCombinedTweetsDeduped.csv.gzip
0403_UkraineCombinedTweetsDeduped.csv.gzip
0404_UkraineCombinedTweetsDeduped.csv.gzip
0405_UkraineCombinedTweetsDeduped.csv.gzip
0406_UkraineCombinedTweetsDeduped.csv.gzip
0407_UkraineCombinedTweetsDeduped.csv.gzip
0408_UkraineCombinedTweetsDeduped.csv.gzip
0409_UkraineCombinedTweetsDeduped.csv.gzip
0410_UkraineCombinedTweetsDeduped.csv.gzip
0411_UkraineCombinedTweetsDeduped.csv.gzip
0412_UkraineCombinedTweetsDeduped.csv.gzip
0413_UkraineCombinedTweetsDeduped.csv.gzip
0414_UkraineCombinedTweetsDeduped.csv.gzip
0415_UkraineCombinedTweetsDeduped.csv.gzip
0416_UkraineCombinedTweetsDeduped.csv.gzip
0417_UkraineCombinedTweetsDeduped.csv.gzip
0418_UkraineCombinedTweetsDeduped.csv.gzip
0419_UkraineCombinedTweetsDeduped.csv.gzip
0420_UkraineCombinedTweetsDeduped.csv.gzip
0421_UkraineCombinedTweetsDeduped.csv.gzip
0422_UkraineCombinedTweetsDeduped.csv.gzip
0423_UkraineCombinedTweetsDeduped.csv.gzip
0424_Ukrain

# Test Small

In [25]:
hg = HashGenerator()
docs = [
    "Lincoln was born into poverty in a log cabin in Kentucky and was raised on the frontier, primarily in Indiana. He was self-educated and became a lawyer, Whig Party leader, Illinois state legislator, and U.S. Congressman from Illinois. In 1849, he returned to his law practice but became vexed by the opening of additional lands to slavery as a result of the Kansas–Nebraska Act of 1854. He reentered politics in 1854, becoming a leader in the new Republican Party, and he reached a national audience in the 1858 Senate campaign debates against Stephen Douglas. Lincoln ran for President in 1860, sweeping the North to gain victory. Pro-slavery elements in the South viewed his election as a threat to slavery, and Southern states began seceding from the Union. During this time the newly formed Confederate States of America began seizing federal military bases in the south. Just over one month after Lincoln assumed the presidency, the Confederate States attacked Fort Sumter, a U.S. fort in South Carolina. Following the bombardment, Lincoln mobilized forces to suppress the rebellion and restore the Union.",
    "Abraham Lincoln was born on February 12, 1809, the second child of Thomas Lincoln and Nancy Hanks Lincoln, in a log cabin on Sinking Spring Farm near Hodgenville, Kentucky.[2] He was a descendant of Samuel Lincoln, an Englishman who migrated from Hingham, Norfolk, to its namesake, Hingham, Massachusetts, in 1638. The family then migrated west, passing through New Jersey, Pennsylvania, and Virginia.[3] Lincoln's paternal grandparents, his namesake Captain Abraham Lincoln and wife Bathsheba (née Herring) moved the family from Virginia to Jefferson County, Kentucky.[b] The captain was killed in an Indian raid in 1786.[5] His children, including eight-year-old Thomas, Abraham's father, witnessed the attack.[6][c] Thomas then worked at odd jobs in Kentucky and Tennessee before the family settled in Hardin County, Kentucky, in the early 1800s.",
    "A supernova is a powerful and luminous stellar explosion. This transient astronomical event occurs during the last evolutionary stages of a massive star or when a white dwarf is triggered into runaway nuclear fusion. The original object, called the progenitor, either collapses to a neutron star or black hole, or is completely destroyed. The peak optical luminosity of a supernova can be comparable to that of an entire galaxy before fading over several weeks or months. Supernovae are more energetic than novae. In Latin, nova means new, referring astronomically to what appears to be a temporary new bright star. Adding the prefix super- distinguishes supernovae from ordinary novae, which are far less luminous. The word supernova was coined by Walter Baade and Fritz Zwicky in 1929.",
    "The most recent directly observed supernova in the Milky Way was Kepler's Supernova in 1604, but the remnants of more recent supernovae have been found. Observations of supernovae in other galaxies suggest they occur in the Milky Way on average about three times every century. These supernovae would almost certainly be observable with modern astronomical telescopes. The most recent naked-eye supernova was SN 1987A, the explosion of a blue supergiant star in the Large Magellanic Cloud, a satellite of the Milky Way."
]

In [57]:
hg = HashGenerator()
docs = [
    "The pen is on the table",
    "The cat is eating something on the table",
    "I watched soccer on television"
]

In [58]:
model = LSHModel(
    k=5,
    threshold=0.5,
    num_hashes=10,
    hash_generator=hg
)

In [59]:
for doc in docs:
    model.add_document(doc)

In [60]:
model.get_similar_pairs()

  0%|          | 3317940/4294967295 [00:50<18:12:46, 65455.25it/s]


KeyboardInterrupt: 

In [46]:
model.check_threshold_on_cm(
    model.candidate_pairs,
    model.docs_dict,
    model.threshold
)

(set(),
 {(0, 1, 0.1956521739130435),
  (0, 2, 0.022727272727272728),
  (1, 2, 0.01639344262295082)})

In [47]:
model.docs_dict

{0: array([ 158808472,  708011900,  919504073,  946367325, 1228039777,
        1537773700, 1543055552, 1609821938, 2126388655, 2199266811,
        2258553440, 3088250546, 3110365843, 3143645696, 3500870551,
        3518176243, 3805288367, 3825420513, 4129918790], dtype=uint32),
 1: array([ 305718398,  351860963,  391691737,  708011900,  727465666,
         800121834,  838362527,  919504073,  921930844, 1026759473,
        1378509836, 1457339044, 1504166779, 1531718787, 1640676111,
        1765300020, 2060798106, 2199266811, 2258553440, 2364073951,
        2565454736, 2810438039, 3088250546, 3110365843, 3143645696,
        3186631732, 3206940652, 3277007011, 3402846202, 3466140034,
        3496598347, 3754242585, 3825420513, 3917732950, 4129918790,
        4194327665], dtype=uint32),
 2: array([ 196352484,  415479271,  579621533,  618866379,  648662924,
         652483296,  670461243, 1216346228, 1237578509, 1332011794,
        1342917158, 1496471397, 1498976282, 1547738922, 1721303408,

In [48]:
model.signature

array([[0, 0, 0],
       [0, 0, 0],
       [0, 0, 0],
       [0, 0, 0],
       [0, 0, 0],
       [0, 0, 0],
       [0, 0, 0],
       [0, 0, 0],
       [0, 0, 0],
       [0, 0, 0]], dtype=uint32)